<a href="https://colab.research.google.com/github/thorblock/Mod_22_SparkView/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# spark_version = 'spark-3.5.0', current working ver
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,277 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,031 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,512 kB]
Hit:13 https://ppa.launchpadconte

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
# define spark, nabs AWS bucket from predestined url
spark.sparkContext.addFile(url)
# use spark def to read specifically our file, .show to make it square-ish
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales_view')


In [14]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# the SQL lingo in orm     choose   col price           col date sold     in   view        filter by bd=4    group by and and then order by date desc
bd4_price_avg = spark.sql("select round(avg(price), 2), year(date) from home_sales_view where bedrooms = 4 group by Year(date) order by year(date) desc")
bd4_price_avg.show(5)


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|           301819.44|      2021|
|           298353.78|      2020|
|            300263.7|      2019|
+--------------------+----------+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
bd3_ba3_build_price_avg = spark.sql("select round(avg(price),2), date_built from home_sales_view where bedrooms = 3 and bathrooms = 3 group by date_built order by date_built desc")
bd3_ba3_build_price_avg.show(5)


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292676.79|      2017|
|           290555.07|      2016|
|            288770.3|      2015|
|           290852.27|      2014|
|           295962.27|      2013|
+--------------------+----------+
only showing top 5 rows



In [15]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
bd3_ba3_f2_sq2000_build_price_avg = spark.sql("select round(avg(price),2), date_built from home_sales_view where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >=2000 group by date_built order by date_built desc")
bd3_ba3_f2_sq2000_build_price_avg.show(5)


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           280317.58|      2017|
|            293965.1|      2016|
|           297609.97|      2015|
|           298264.72|      2014|
|           303676.79|      2013|
+--------------------+----------+
only showing top 5 rows



In [27]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
view_350k = spark.sql("select view, round(avg(price),2) from home_sales_view group by view having avg(price) >= 350000 order by view desc")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.05784153938293457 seconds ---


In [29]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales_view")


DataFrame[]

In [30]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_view')

True

In [31]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()
view_350k_cached = spark.sql("select view, round(avg(price),2) from home_sales_view group by view having avg(price) >= 350000 order by view desc")

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.031081676483154297 seconds ---


In [33]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.parquet("home_parquet", mode="overwrite")

In [34]:
# 11. Read the parquet formatted data.
home_sales_parquet = spark.read.parquet("home_parquet")

In [36]:
# 12. Create a temporary table for the parquet data.
home_sales_parquet.createOrReplaceTempView("home_sales_parquet")
home_sales_parquet.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [37]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time = time.time()
view_350k_parquet = spark.sql("select view, round(avg(price),2) from home_sales_view group by view having avg(price) >= 350000 order by view desc")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.03802800178527832 seconds ---


In [39]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales_view")

DataFrame[]

In [40]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales_view')

False